In [1]:
import cv2
import os, glob
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
from dataset import *
import pandas as pd
from PIL import Image
from torchvision import transforms
from torchvision import models as tm
from efficientnet_pytorch import EfficientNet
from model import *
from datetime import datetime
from torch.nn.parallel import DataParallel
import tqdm

##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device='cuda'
now = datetime.now()
# dd/mm/YY
val_batch_size=256
root = "/ssd3/ilsvrc2012/"
val_fn = root+'val.txt'


stmodel= tm.resnet101(pretrained=True).to(device)
imTrs = imageTrans(device).to(device)
stmodel = DataParallel(stmodel)
imTrs = DataParallel(imTrs)

stmodel.load_state_dict(torch.load('checkpoint/final_net.pt'))
imTrs.load_state_dict(torch.load('checkpoint/final_imt.pt'))

val_dataset = dataset_DFD(0, val_fn, root=root+'val/', mode='val')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=val_batch_size, drop_last=True, 
                                         num_workers=16, sampler=None, shuffle=False)

stmodel.eval()
val_acc = 0
print('Start validating')
opt_inner = torch.optim.AdamW(imTrs.parameters(), lr=3e-5)
mse=torch.nn.MSELoss().type(torch.cuda.FloatTensor)

val_acc = []
for X,y in tqdm.tqdm(val_loader, total=len(val_loader)):
    X = X.to(device)
    y = y.to(device)
    ## Inner optimization====
    imTrs.train()
    for inner_ep in range(4):
        opt_inner.zero_grad()
        X_in = imTrs(X).to(device)
        loss_inner = mse(X_in, X)
        loss_inner.backward()
        opt_inner.step()
    imTrs.eval()
    ##======================
    with torch.no_grad():
        pred = stmodel(imTrs(X))
        _, pred = torch.max(pred, 1)
        val_acc.append( (pred == y).sum().item()/y.size(0))

val_acc = np.mean(np.array(val_acc))

now = datetime.now()
d1 =now.strftime("%d/%m/%Y %H:%M:%S")
print("[%s] Validation Accuracy=%.5f" % (d1, val_acc))


  0%|          | 0/195 [00:00<?, ?it/s]

The # of images is: 50000 on val mode!
Start validating


100%|██████████| 195/195 [07:30<00:00,  2.31s/it]


NameError: name 'step' is not defined

[02/01/2021 12:09:49] Validation Accuracy=0.74283
